<a href="https://colab.research.google.com/github/Bhardwaj-Saurabh/Agents_Design_Pattern/blob/master/reflection_pattern_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -U langchain-google-genai
# !pip install colorama

In [2]:
import os
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [3]:
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from colorama import init, Fore, Style

init(autoreset=True)

In [6]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
)

In [7]:
messages = [
    (
        "system",
        """You are an expert python programmer.
            You are tasked to create best possible python program based on the user request.
            Your output must only be the python code and nothing else.""",
    ),
]


In [8]:
messages.append(
    ("human",
     "Create the hangman game for user with medium difficulty level.")
)

In [9]:
response = llm.invoke(messages)

In [10]:
generated_code = response.content

In [11]:
from IPython.display import Markdown
Markdown(response.content)

```python
import random

def choose_word():
    """Chooses a random word from a predefined list."""
    words = ["python", "programming", "computer", "science", "algorithm", "function", "variable", "database", "developer", "keyboard"]
    return random.choice(words)

def display_word(word, guessed_letters):
    """Displays the word with correctly guessed letters and underscores for unguessed letters."""
    displayed_word = ""
    for letter in word:
        if letter in guessed_letters:
            displayed_word += letter
        else:
            displayed_word += "_"
    return displayed_word

def hangman():
    """Runs the hangman game."""
    word_to_guess = choose_word()
    guessed_letters = set()
    attempts_left = 6  # You can adjust the number of attempts for difficulty
    
    print("Welcome to Hangman!")
    print(display_word(word_to_guess, guessed_letters))
    print(f"You have {attempts_left} attempts left.")

    while attempts_left > 0:
        guess = input("Guess a letter: ").lower()

        if not guess.isalpha() or len(guess) != 1:
            print("Invalid input. Please enter a single letter.")
            continue

        if guess in guessed_letters:
            print("You already guessed that letter.")
            continue

        guessed_letters.add(guess)

        if guess in word_to_guess:
            print("Correct guess!")
            displayed = display_word(word_to_guess, guessed_letters)
            print(displayed)
            if "_" not in displayed:
                print("Congratulations! You guessed the word:", word_to_guess)
                return
        else:
            attempts_left -= 1
            print("Incorrect guess.")
            print(f"You have {attempts_left} attempts left.")
            print(display_word(word_to_guess, guessed_letters))

        if attempts_left == 0:
            print("You ran out of attempts.")
            print("The word was:", word_to_guess)

if __name__ == "__main__":
    hangman()
```

In [12]:
critique_messages = [
    (
        "system",
        """You are an expert python programmer.
            You are tasked to critique the given python code.
            Provide your critiques in bullet points.
            Provide recommendation to improve the quality of the code.
            Do not provide the code. Only critique and recommendations.""",
    ),
    ("human", generated_code)
]

In [13]:
critique = llm.invoke(critique_messages)

In [14]:
critique_output = critique.content

In [15]:
Markdown(critique.content)

Here's a critique of the provided Python code, along with recommendations for improvement:

**Critiques:**

*   **Limited Word List:** The `choose_word` function has a hardcoded list of words. This limits the replayability of the game.
*   **Lack of Visual Hangman:** The game lacks a visual representation of the hangman figure, which is a common and engaging element of the game.
*   **No Input Validation for Non-Letter Guesses:** While the code checks for single-character input, it doesn't explicitly prevent the user from entering numbers or special characters (although `.isalpha()` handles this). A more explicit check might improve clarity.
*   **Repetitive Output:** The `display_word` function is called multiple times within the loop, leading to some repetition in the output.
*   **Game Logic in `hangman` Function:** The `hangman` function is quite long and contains a mix of game logic, input/output, and display elements. This makes it harder to read and maintain.
*   **No Option to Play Again:** The game ends after one round, without offering the player the option to play again.
*   **Missing Docstrings:** While some functions have docstrings, the `hangman` function itself could benefit from a more detailed docstring explaining its overall purpose and behavior.

**Recommendations:**

*   **Expand Word List:**
    *   Load words from a file (e.g., a text file containing a list of words, one word per line). This makes it easy to add more words without modifying the code.
*   **Implement Visual Hangman:**
    *   Create a list of strings representing the different stages of the hangman figure. Display the appropriate stage based on the number of attempts remaining.  Consider using ASCII art for a simple visual.
*   **Improve Input Validation:**
    *   While `.isalpha()` works, consider adding a more explicit check and a more informative error message if the input is not a letter.
*   **Reduce Repetitive Output:**
    *   Store the displayed word in a variable and only update the display when it changes.
*   **Refactor `hangman` Function:**
    *   Break down the `hangman` function into smaller, more manageable functions. For example:
        *   A function to get the player's guess.
        *   A function to update the game state based on the guess.
        *   A function to check if the game is won or lost.
*   **Add "Play Again" Option:**
    *   After the game ends (win or lose), ask the player if they want to play again. Wrap the game logic in a `while` loop to allow multiple games.
*   **Add More Detailed Docstrings:**
    *   Provide comprehensive docstrings for all functions, including the `hangman` function, explaining their purpose, parameters, and return values.
*   **Consider Using Constants:**
    *   Define the number of attempts as a constant at the beginning of the script (e.g., `MAX_ATTEMPTS = 6`). This makes it easier to change and improves readability.
*   **Handle Edge Cases:**
    *   Consider what happens if the word list is empty. Add a check to handle this case gracefully.
*   **Improve User Experience:**
    *   Clear the screen after each guess (using `os.system('cls')` on Windows or `os.system('clear')` on Linux/macOS) to make the game more visually appealing.  Be mindful of cross-platform compatibility.
*   **Consider Exception Handling:**
    *   If you load words from a file, use `try...except` blocks to handle potential `IOError` exceptions if the file cannot be opened or read.

In [16]:
final_messages = [
    (
        "system",
        """You are an expert python programmer.
            You are tasked is to improve the given code based on provided recommendations and critique by senior expert.
            Your output must only the the improved code.
            Do not inlcude anything else in theoutput.""",
    ),
    ("human", generated_code + '\n\n' +critique_output )
]

In [17]:
final_code = llm.invoke(final_messages)

In [18]:
final_code_output = final_code.content

In [19]:
Markdown(final_code_output)

```python
import random
import os

MAX_ATTEMPTS = 6  # Define the maximum number of attempts
HANGMAN_STAGES = [  # Visual representation of the hangman
    """
      +---+
      |   |
          |
          |
          |
          |
    =========""",
    """
      +---+
      |   |
      O   |
          |
          |
          |
    =========""",
    """
      +---+
      |   |
      O   |
      |   |
          |
          |
    =========""",
    """
      +---+
      |   |
      O   |
     /|   |
          |
          |
    =========""",
    """
      +---+
      |   |
      O   |
     /|\  |
          |
          |
    =========""",
    """
      +---+
      |   |
      O   |
     /|\  |
     /    |
          |
    =========""",
    """
      +---+
      |   |
      O   |
     /|\  |
     / \  |
          |
    ========="""]


def choose_word(filepath="words.txt"):
    """Chooses a random word from a file.

    Args:
        filepath (str, optional): The path to the file containing the list of words.
            Defaults to "words.txt".

    Returns:
        str: A randomly chosen word from the file, or None if the file is empty or cannot be read.
    """
    try:
        with open(filepath, "r") as f:
            words = [line.strip() for line in f]
        if not words:
            print("Word list file is empty.")
            return None
        return random.choice(words)
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    except IOError:
        print(f"Error: Could not read file at {filepath}")
        return None


def display_word(word, guessed_letters):
    """Displays the word with correctly guessed letters and underscores for unguessed letters.

    Args:
        word (str): The word to be displayed.
        guessed_letters (set): A set of letters that have been guessed correctly.

    Returns:
        str: The displayed word with correctly guessed letters and underscores.
    """
    displayed_word = ""
    for letter in word:
        if letter in guessed_letters:
            displayed_word += letter
        else:
            displayed_word += "_"
    return displayed_word


def get_player_guess():
    """Gets a letter guess from the player with input validation.

    Returns:
        str: The player's guess (a single lowercase letter), or None if the input is invalid.
    """
    guess = input("Guess a letter: ").lower()

    if not guess.isalpha() or len(guess) != 1:
        print("Invalid input. Please enter a single letter.")
        return None
    return guess


def update_game_state(word, guessed_letters, guess, attempts_left):
    """Updates the game state based on the player's guess.

    Args:
        word (str): The word to be guessed.
        guessed_letters (set): The set of already guessed letters.
        guess (str): The player's current guess.
        attempts_left (int): The number of attempts remaining.

    Returns:
        tuple: A tuple containing the updated guessed_letters (set), attempts_left (int),
               and a boolean indicating whether the guess was correct (bool).
    """
    if guess in guessed_letters:
        print("You already guessed that letter.")
        return guessed_letters, attempts_left, False

    guessed_letters.add(guess)

    if guess not in word:
        attempts_left -= 1
        print("Incorrect guess.")
        return guessed_letters, attempts_left, False

    print("Correct guess!")
    return guessed_letters, attempts_left, True


def check_game_status(word, guessed_letters, attempts_left):
    """Checks if the game is won or lost.

    Args:
        word (str): The word to be guessed.
        guessed_letters (set): The set of letters guessed so far.
        attempts_left (int): The number of attempts remaining.

    Returns:
        str: "win" if the game is won, "lose" if the game is lost, or None if the game is still in progress.
    """
    displayed = display_word(word, guessed_letters)
    if "_" not in displayed:
        return "win"
    if attempts_left == 0:
        return "lose"
    return None


def play_hangman():
    """Plays a single round of Hangman."""
    word_to_guess = choose_word()
    if word_to_guess is None:
        return  # Exit if no word could be chosen

    guessed_letters = set()
    attempts_left = MAX_ATTEMPTS
    game_over = False

    print("Welcome to Hangman!")
    displayed_word = display_word(word_to_guess, guessed_letters)
    print(displayed_word)
    print(f"You have {attempts_left} attempts left.")
    print(HANGMAN_STAGES[0])  # Initial hangman stage

    while not game_over:
        guess = get_player_guess()
        if guess is None:
            continue  # Invalid input, get another guess

        guessed_letters, attempts_left, correct_guess = update_game_state(
            word_to_guess, guessed_letters, guess, attempts_left
        )

        displayed_word = display_word(word_to_guess, guessed_letters)
        print(displayed_word)
        print(f"You have {attempts_left} attempts left.")
        print(HANGMAN_STAGES[MAX_ATTEMPTS - attempts_left])  # Update hangman visual

        game_status = check_game_status(word_to_guess, guessed_letters, attempts_left)

        if game_status == "win":
            print("Congratulations! You guessed the word:", word_to_guess)
            game_over = True
        elif game_status == "lose":
            print("You ran out of attempts.")
            print("The word was:", word_to_guess)
            game_over = True


def hangman():
    """Runs the Hangman game, allowing multiple rounds."""
    play_again = True
    while play_again:
        play_hangman()
        answer = input("Play again? (yes/no): ").lower()
        if answer != "yes":
            play_again = False
    print("Thanks for playing!")


if __name__ == "__main__":
    hangman()
```

In [20]:
def format_prompt_structure(prompt: str, role: str) -> tuple:
    return (role, prompt)

In [26]:
class ReflectionAgent:
    def __init__(self, model_name="gemini-2.0-flash", temperature=0.1):
        self.model = ChatGoogleGenerativeAI(
            model=model_name,
            temperature=temperature
        )

    def generate_output(self, messages):
        return self.model.invoke(messages)

    def run(self, user_message, steps=3, verbose=True):
        system_prompt = format_prompt_structure(
            """You are an expert python programmer.
            You are tasked to create best possible python program based on the user request.
            If provided with critique and recommendation, please use those to improve the code.
            Your output must only be the python code and nothing else.""", "system")

        critique_message = """You are an expert python programmer.
            You are tasked to critique the given python code.
            Provide your critiques in bullet points.
            Provide recommendation to improve the quality of the code.
            Do not provide the code. Only critique and recommendations.

            If Finished is True, Just reply with one word --> Finished.
            FINISHED: """

        user_prompt = format_prompt_structure(user_message, "user")
        messages = [system_prompt, user_prompt]

        FINISH = False

        for step in range(steps):
            if step == steps - 1:
                FINISH = True

            if verbose:
                print(Fore.CYAN + f"\n🔁 Iteration {step + 1}/{steps}")

            code_response = self.generate_output(messages)
            messages.append(code_response)

            if verbose:
                print(Fore.YELLOW + "\n🧾 Generated Code:")
                print(Fore.GREEN + code_response.content)

            critique_prompt = format_prompt_structure(critique_message + " " + str(FINISH), "system")

            messages.append(critique_prompt)
            critique_response = self.generate_output(messages)

            if verbose:
                print(Fore.YELLOW + "\n🧐 Critique:")
                print(Fore.MAGENTA + critique_response.content)

            if "finished" in critique_response.content.lower():
                if verbose:
                    print(Fore.BLUE + "\n✅ Finished early as critique said 'Finished'")
                break

            messages.append(critique_response)

        if verbose:
            print(Fore.CYAN + "\n🎯 Final Code or Critique chain completed.")

        return messages



In [27]:
agent = ReflectionAgent()

In [28]:
response = agent.run("write a hangman game")


🔁 Iteration 1/3

🧾 Generated Code:
```python
import random

def choose_word():
    """Chooses a random word from a predefined list."""
    words = ["python", "hangman", "programming", "computer", "science", "keyboard", "algorithm"]
    return random.choice(words)

def display_word(word, guessed_letters):
    """Displays the word with correctly guessed letters and underscores for unguessed letters."""
    displayed_word = ""
    for letter in word:
        if letter in guessed_letters:
            displayed_word += letter
        else:
            displayed_word += "_"
    return displayed_word

def hangman():
    """Plays a game of hangman."""
    word_to_guess = choose_word()
    guessed_letters = set()
    attempts_left = 6

    print("Welcome to Hangman!")
    print(display_word(word_to_guess, guessed_letters))
    print(f"You have {attempts_left} attempts left.")

    while attempts_left > 0:
        guess = input("Guess a letter: ").lower()

        if not guess.isalpha() or len(

In [29]:
for i in response:
    print(i)

('system', 'You are an expert python programmer. \n            You are tasked to create best possible python program based on the user request.\n            If provided with critique and recommendation, please use those to improve the code.\n            Your output must only be the python code and nothing else.')
('user', 'write a hangman game')
content='```python\nimport random\n\ndef choose_word():\n    """Chooses a random word from a predefined list."""\n    words = ["python", "hangman", "programming", "computer", "science", "keyboard", "algorithm"]\n    return random.choice(words)\n\ndef display_word(word, guessed_letters):\n    """Displays the word with correctly guessed letters and underscores for unguessed letters."""\n    displayed_word = ""\n    for letter in word:\n        if letter in guessed_letters:\n            displayed_word += letter\n        else:\n            displayed_word += "_"\n    return displayed_word\n\ndef hangman():\n    """Plays a game of hangman."""\n    wo

In [30]:
Markdown(response[-3].content)

```python
import random

def choose_word():
    """Chooses a random word from a predefined list."""
    words = ["python", "hangman", "programming", "computer", "science", "keyboard", "algorithm"]
    return random.choice(words)

def display_word(word, guessed_letters):
    """Displays the word with correctly guessed letters and underscores for unguessed letters."""
    current_display = ""
    for letter in word:
        if letter in guessed_letters:
            current_display += letter
        else:
            current_display += "_"
    return current_display

def display_hangman(attempts_left):
    """Displays the hangman figure based on the number of attempts left."""
    stages = [
        """
           --------
           |      |
           |      O
           |     \\|/
           |      |
           |     / \\
           -
        """,
        """
           --------
           |      |
           |      O
           |     \\|/
           |      |
           |     / 
           -
        """,
        """
           --------
           |      |
           |      O
           |     \\|/
           |      |
           |      
           -
        """,
        """
           --------
           |      |
           |      O
           |     \\|
           |      |
           |      
           -
        """,
        """
           --------
           |      |
           |      O
           |      |
           |      |
           |      
           -
        """,
        """
           --------
           |      |
           |      O
           |      
           |      
           |      
           -
        """,
        """
           --------
           |      |
           |      
           |      
           |      
           |      
           -
        """,
    ]
    return stages[6 - attempts_left]


def hangman():
    """Plays a game of hangman."""
    word_to_guess = choose_word()
    guessed_letters = set()
    attempts_left = 6
    
    print("Welcome to Hangman!")
    print(display_hangman(attempts_left))
    print(display_word(word_to_guess, guessed_letters))
    print(f"You have {attempts_left} attempts left.")

    while attempts_left > 0:
        guess = input("Guess a letter: ").lower()

        if not guess.isalpha() or len(guess) != 1:
            print("Invalid input. Please enter a single letter.")
            continue

        if guess in guessed_letters:
            print("You already guessed that letter.")
            continue

        guessed_letters.add(guess)

        if guess in word_to_guess:
            print("Correct guess!")
            current_display = display_word(word_to_guess, guessed_letters)
            print(current_display)

            if "_" not in current_display:
                print("Congratulations! You guessed the word:", word_to_guess)
                return
        else:
            attempts_left -= 1
            print("Incorrect guess.")
            print(display_hangman(attempts_left))
            print(display_word(word_to_guess, guessed_letters))
            print(f"You have {attempts_left} attempts left.")

    print("You ran out of attempts. The word was:", word_to_guess)

    play_again = input("Do you want to play again? (yes/no): ").lower()
    if play_again == "yes":
        hangman()

if __name__ == "__main__":
    hangman()
```